In [47]:
import modules.swmm_ex.swmm_ex_multiprocessing as ex
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os
from modules.extract_events import extract_events
from modules.swmm_ex.swmm_ex_multiprocessing import swmm_mp

# Set relevant data paths
events_path = '02_input_data\\events_FMO.csv'
P_path = '02_input_data\\P_FMO.csv'
save_folder = '03_sim_data\\quick_sim'

# Read in data
P_events = pd.read_csv(events_path)
P_series = pd.read_csv(P_path)


P_sample = P_events[(P_events['hN_mm'] < 2.5) & (P_events['hN_mm'] > 1)].sample(5,random_state=5)

P_sample

,start,end,hN_mm,iN_mean,iN_max,hN_max_60,DN_h
177,2015-01-15 04:25:00,2015-01-15 14:35:00,2.34,0.019024,0.07,0.54,10.166667
1508,2022-10-20 12:45:00,2022-10-20 14:40:00,1.43,0.059583,0.30,0.99,1.916667
1509,2022-10-23 17:25:00,2022-10-23 18:30:00,2.01,0.143571,0.43,1.98,1.083333
513,2016-10-28 14:50:00,2016-10-28 17:15:00,1.47,0.049000,0.36,1.35,2.416667
474,2016-07-12 17:15:00,2016-07-12 17:35:00,1.61,0.322000,0.61,0.00,0.333333


In [48]:
extract_events(P_sample, P_series, save_folder)

Event 2015-01-15 04:25:00 saved to 03_sim_data\quick_sim\2015-01-15 04 25 00_hN2.34.csv
Event 2022-10-20 12:45:00 saved to 03_sim_data\quick_sim\2022-10-20 12 45 00_hN1.43.csv
Event 2022-10-23 17:25:00 saved to 03_sim_data\quick_sim\2022-10-23 17 25 00_hN2.01.csv
Event 2016-10-28 14:50:00 saved to 03_sim_data\quick_sim\2016-10-28 14 50 00_hN1.47.csv
Event 2016-07-12 17:15:00 saved to 03_sim_data\quick_sim\2016-07-12 17 15 00_hN1.61.csv


In [49]:
import swmm_api
from swmm_api.input_file import section_labels as sections
from modules.inp_helpers import euler_to_inp, event_to_inp

# get current path of working directory
current_path = os.getcwd()
###########################################################################################################
# Input section
## Input parameters for inp file generation
base_inp_path = '03_sim_data\\swmm_Gievenbeck.inp'
# path to folder with rain event data
event_data_path = save_folder
# path to kostra data
kostra_data_path = os.path.join(current_path, '02_input_data\\kostra_118111.csv')
# set maximum duration time [min] for Kostra data
max_duration = 72*60
# Name of the study area
name_place = 'Gievenbeck'
# Path to save the inp files
save_inp_path = save_folder
# Euler type for Kostra data (2 is standard)
euler_typ = 2
# Start time of the simulation
start_time = pd.to_datetime('2024-01-01 00:00')
# Buffer time before and after the rainfall event
buffer_time = pd.Timedelta('2h')
# Name of the Kostra time series to be included in the inp file
TSnameKostra = 'Kostra'
# Name of the measured time series to be included in the inp file
TSnameEvent = 'FMO'
# Time interval of the time series in minutes
TSinterval = 5

########################
# read kostra data
kostra = pd.read_csv(kostra_data_path, delimiter=',', index_col=0)
# get return preiods and durations from kostra table
returnrate = kostra.columns.astype(int)
# duration needs to be larger than 15min
durations = kostra.index[(kostra.index >= 15) & (kostra.index <= max_duration)]
# calculate end time of the simulation with start time and buffer time
end_time = start_time + pd.Timedelta(minutes=int(max(durations))) + buffer_time * 2

########################
# read base inp file
base_inp_path = '03_sim_data\\swmm_Gievenbeck.inp'
inp_base = swmm_api.read_inp_file(base_inp_path)
# Update OPTIONS of inp file
inp_base['OPTIONS'].update({'START_DATE': start_time.date()})
inp_base['OPTIONS'].update({'START_TIME': start_time.time()})
inp_base['OPTIONS'].update({'REPORT_START_DATE': start_time.date()})
inp_base['OPTIONS'].update({'REPORT_START_TIME': start_time.time()})
inp_base['OPTIONS'].update({'END_DATE': end_time.date()})
inp_base['OPTIONS'].update({'END_TIME': end_time.time()})
# inp_base['OPTIONS'].update({'THREADS': cpu_cores})

In [50]:
inp = inp_base
try:
    del inp[sections.TIMESERIES][TSnameKostra]
except Exception:
    pass
try:
    del inp['RAINGAGES'][TSnameKostra]
except Exception:
    pass

for file_name in os.listdir(event_data_path):
    if file_name.endswith('.csv'):
        
        file_path = os.path.join(event_data_path, file_name)
        event_data = pd.read_csv(file_path)
        inp = event_to_inp(inp, event_data, start_time=start_time + buffer_time, TSname=TSnameEvent)
        for subcatchment in inp['SUBCATCHMENTS']:
            inp['SUBCATCHMENTS'][subcatchment].rain_gage = TSnameEvent
        file_name = file_name.replace('.csv', '')
        file_name = file_name.replace('.', ' ')
        inp['TITLE'] = f'{name_place}_{file_name}'
        inp.write_file(os.path.join(save_inp_path,f'{name_place}_{file_name}.inp'))

In [51]:
swmm_mp(save_folder)



Total simulation time:  9.892069101333618
All done


#### Simulation: Gievenbeck_2015-01-15 04 25 00_hN2 34
Niederschlagshöhe: 2.34
Abflusshöhe: 0.1 mm

#### Simulation: Gievenbeck_2022-10-23 17 25 00_hN2 01
Niederschlagshöhe: 2.01
Abflusshöhe: 0.1 mm

Alle Simulationen mit weniger Niederschlaghöhe erzeugnten keinen Abfluss.
Daher werden nur Events mit mehr als 2 mm Niederschlaghöhe ausgewählt
